In [23]:
_input_path = experiment_data_dir / "evidence" / "combined_score.json"
with _input_path.open() as f:
    combined_evidence = json.load(f)
print(len(combined_evidence))

413


In [24]:
_input_path = analysis_dir / "terms_match_epigraphdb.json"
with _input_path.open() as f:
    terms_match_epigraphdb = json.load(f)
print(len(terms_match_epigraphdb))
print(terms_match_epigraphdb[0])

413
{'doi': '10.1101/19001719', 'subject_term': 'Antihypertensive Agents', 'object_term': 'Mood Disorders', 'subject_term_match': {'efo': [], 'gwas': [], 'literature_term': [{'n': {'_name': 'Antihypertensive Agents', 'name': 'Antihypertensive Agents', '_source': ['SemMedDB_VER42_2020_R', 'bioRxiv-2020-10-06', 'medRxiv-2020-10-06'], 'id': 'C0003364', '_id': 'C0003364', 'type': ['phsu']}}]}, 'object_term_match': {'efo': [], 'gwas': [], 'literature_term': []}}


----

filter evidence where the associated subject and object entities 
have been identified by epigraphdb direct search

In [37]:
def _filter_assoc_evidence(evidence_list, subject_term_matches, object_term_matches):
    res = (
        py_.chain(evidence_list)
        .filter(
            lambda e: e["subject_id"] in subject_term_matches
            and e["object_id"] in object_term_matches
        )
        .value()
    )
    return res


def _filter_triple_evidence(evidence_list, subject_term_matches, object_term_matches):
    res = (
        py_.chain(evidence_list)
        .filter(
            lambda e: e["triple_subject_id"] in subject_term_matches
            and e["triple_object_id"] in object_term_matches
        )
        .value()
    )
    return res


def _filter_combined_evidence(terms_match_entry, evidence_entry):
    # gwas terms and assoc evidence
    gwas_subject_term_matches = (
        py_.chain(terms_match_entry["subject_term_match"]["gwas"])
        .map(lambda e: e["n"]["_id"])
        .value()
    )
    gwas_object_term_matches = (
        py_.chain(terms_match_entry["object_term_match"]["gwas"])
        .map(lambda e: e["n"]["_id"])
        .value()
    )
    assoc_evidence = evidence_entry["assoc_evidence"]
    assoc_evidence_filter = {
        _: _filter_assoc_evidence(
            assoc_evidence[_], gwas_subject_term_matches, gwas_object_term_matches
        )
        for _ in assoc_evidence.keys()
    }
    # literature terms and literature triple evidence
    lit_subject_term_matches = (
        py_.chain(terms_match_entry["subject_term_match"]["literature_term"])
        .map(lambda e: e["n"]["_id"])
        .value()
    )
    lit_object_term_matches = (
        py_.chain(terms_match_entry["object_term_match"]["literature_term"])
        .map(lambda e: e["n"]["_id"])
        .value()
    )
    triple_evidence = evidence_entry["triple_evidence"]
    triple_evidence_filter = {
        _: _filter_triple_evidence(
            triple_evidence[_], lit_subject_term_matches, lit_object_term_matches
        )
        for _ in triple_evidence.keys()
    }
    # combined res
    res = {
        "doi": evidence_entry["doi"],
        "subject_term": evidence_entry["subject_term"],
        "object_term": evidence_entry["object_term"],
        "pred_term": evidence_entry["pred_term"],
        "assoc_evidence": assoc_evidence_filter,
        "triple_evidence": triple_evidence_filter,
    }
    return res


combined_evidence_filter_epigraphdb = (
    py_.chain(range(len(terms_match_epigraphdb)))
    .map(
        lambda idx: _filter_combined_evidence(
            terms_match_entry=terms_match_epigraphdb[idx],
            evidence_entry=combined_evidence[idx],
        )
    )
    .value()
)

In [38]:
_output_path = analysis_dir / "combined_evidence_filter_epigraphdb.json"
with _output_path.open("w") as _f:
    json.dump(combined_evidence_filter_epigraphdb, _f)

----

In [ ]:
TODO: load back evidence_filter_epigraphdb

----

In [ ]:
TODO: asq thresholds

----

In [ ]:
TODO: filter evidence plot